<a href="https://colab.research.google.com/github/Madalina0303/Licenta-Depresie/blob/main/Alticlasificatori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from xml.dom.minidom import parse
import os
import re
import string
import numpy as np
import pandas as pd
import math
from nltk.corpus import stopwords

In [2]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "i'd": "i would", "i'd've": "i would have","i'll": "i will",
                     "i'll've": "i will have","i'm": "i am","I've": "i have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

In [3]:
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])


In [6]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [7]:
def parse_file(filename):
    doc = parse(filename, )
    titles = doc.getElementsByTagName("TITLE")
    posts = doc.getElementsByTagName("TEXT")
    messages = []
    for index, title in enumerate(titles):
        message = title.firstChild.nodeValue + posts[index].firstChild.nodeValue
        message = expand_contractions(message).lower()
        message= re.sub('[%s]' % re.escape(string.punctuation), '' , message)
        message = re.sub('W*dw*','',message)
        message = remove_stopwords(message)
        message = re.sub('(http[s]?S+)|(w+.[A-Za-z]{2,4}S*)', '', message)
        message = lemmatize_words(message)
        message  = re.sub(' +', ' ', message)
        messages.append(message.strip())
        
    return messages

def get_data_post(dir_name):
    users_posts = dict()
    for root, dir, files in os.walk(dir_name):
        for filename in files:
            if ".xml" in filename:
                nr = re.findall("[0-9]+", filename)
                users_posts[nr[0]] = parse_file(root+"/"+filename)

    return users_posts

In [ ]:
 posts = get_data_post("drive/MyDrive/data")
#  print(posts)

In [8]:
 from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# dam append sa facem un vector mare din toate postarile, oare cum ar fi cu o matrice ??

In [9]:
def get_data_post_list(dir_name):
    users_posts_list = []
    for root, dir, files in os.walk(dir_name):
        for filename in files:
            if ".xml" in filename:
                nr = re.findall("[0-9]+", filename)
                users_posts_list.append(parse_file(root+"/"+filename))

    return users_posts_list

In [10]:
all_data = get_data_post_list("drive/MyDrive/data")
x_train = all_data[:60]
x_test = all_data[60:]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000)
 # Stop words are common words in English that don't tell us anything about the polarity of a review.
    # Such words include the, that, and a
# Converts a collection of text documents to a matrix of token counts
# max_features = maximum number of words we'd like to have in our bag of words model
X = []
for i in all_data:
  X.append(cv.fit_transform(i).toarray())

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer()
X = tf_transformer.fit_transform(X).toarray()

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(max_features =1000)
X = []
for i in all_data:
  X.append(tfidfVectorizer.fit_transform(i).toarray())
  


In [26]:
vectorizer = TfidfVectorizer(max_features = 5000) 
for i in all_data:
  vectorizer.fit(i) 
  X.append(tfidfVectorizer.transform(i).toarray())

# Xtrain = vectorizer.transform(all_data)
print(len(X[0]))
print(len(X[1]))

73
43


In [19]:
print(len(X[1]))
!pip install catboost

43
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 103 kB/s 


In [24]:
print(X[1])

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.36931339 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.18000993 0.         0.        ]]


In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

# Naive Bayes is a statistical classification technique based on Bayes Theorem
# common classifier used in sentiment analysis is the Naive Bayes Classifier.
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier # this is experimental
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
classifiers = [GradientBoostingClassifier(),GaussianNB(),HistGradientBoostingClassifier(),
               RandomForestClassifier(),LogisticRegression(),XGBClassifier(),LGBMClassifier(),
               CatBoostClassifier(verbose=0)]

In [32]:
import json
with open('drive/MyDrive/Licenta/reponses.json') as json_file:
    responses = json.load(json_file)
  
y_train = responses["Sadness"]
import numpy as np
# one hot target 
# momentan avem 4 raspunsuri de la 0 la 3 facem cu I4 la celelalte mai speciale facem cu I8
print(y_train)
# trg = [[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,0.0,1.0]]
# y_train_target = []
# for j in y_train:
#   y_train_target.append(trg[j])
# y_1 = np.array(y_train_target)
# print(type(y_train_target[0][0]))

y_test = y_train[60:]
y_train = y_train[0:60]

[1, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 0, 1, 0, 3, 1, 1, 1, 0, 3, 2, 1, 1, 3, 1, 1, 1, 0, 1, 1, 3, 1, 3, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1, 2, 2, 1, 1, 3, 1, 0, 1, 1, 2, 2, 0, 1, 0, 0, 3, 1, 1, 0, 1, 1, 1, 3, 2, 3, 1]


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
for classifier in classifiers:
    classifier.fit(x_train,y_train)
    print(f'The {classifier}  Accuracy  is {accuracy_score(y_test,classifier.predict(X)) }' )



```
# Acesta are format de cod
```



In [27]:

from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 100000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x_train)

In [28]:
x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)


In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [30]:
# max_length = 150530
padding_type = "post"
trunction_type="post"
x_train_padded = pad_sequences(x_train_sequences, padding=padding_type,
                       truncating=trunction_type,maxlen = 1218)
x_test_padded = pad_sequences(x_test_sequences,
                               padding=padding_type, truncating=trunction_type,maxlen = 1218)

In [34]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
for classifier in classifiers:
    classifier.fit(x_train_padded,y_train)
    print(f'The {classifier}  Accuracy  is {accuracy_score(y_test,classifier.predict(x_test_padded )) }' )

The GradientBoostingClassifier()  Accuracy  is 0.45
The GaussianNB()  Accuracy  is 0.05
The HistGradientBoostingClassifier()  Accuracy  is 0.45
The RandomForestClassifier()  Accuracy  is 0.45


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


The LogisticRegression()  Accuracy  is 0.2
The XGBClassifier(objective='multi:softprob')  Accuracy  is 0.45
The LGBMClassifier()  Accuracy  is 0.45
The <catboost.core.CatBoostClassifier object at 0x7f619f5fe550>  Accuracy  is 0.45


In [ ]:
print(type(x_train_padded))
print(x_train_padded.shape)
print(x_test_padded.shape)

<class 'numpy.ndarray'>
(60, 1218)
(20, 1218)


In [ ]:
!wget --no-check-certificate \
http://nlp.stanford.edu/data/glove.6B.zip \
-O /tmp/glove.6B.zip

In [ ]:
import os
import zipfile
with zipfile.ZipFile('/tmp/glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/glove')

In [ ]:
lg = 0
import numpy as np
embeddings_index = {}
f = open('/tmp/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    if len(coefs>lg):
      lg = len(coefs)
    embeddings_index[word] = coefs
f.close()
print(lg)
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
word_index = {w: i for i, w in enumerate(embeddings_index.keys(), 1)}

In [ ]:
import numpy as np
embedding_matrix = np.zeros((len(word_index) + 1,100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D

In [ ]:
embedding_layer = Embedding(input_dim=len(word_index) + 1,
                            output_dim=100,
                            weights=[embedding_matrix],
                            input_length=1218,
                            trainable=False)

In [ ]:
model = Sequential([
    embedding_layer,
  Conv1D(256, 10, activation='relu'),
    GlobalMaxPooling1D(),
  Dense(500, activation='relu'),
  Dense(100, activation='relu'),
  Dense(10, activation='relu'),
  Dense(4, activation='softmax')
])

In [ ]:
# from tensorflow import reduce_mean,reduce_sum
# loss1= reduce_mean(reduce_sum(cross_entropy, axis=1))
model.compile(loss= "ce",optimizer='adam',metrics=['accuracy'])

In [21]:
import json
with open('drive/MyDrive/Licenta/reponses.json') as json_file:
    responses = json.load(json_file)
  
y_train = responses["Sadness"]
import numpy as np
# one hot target 
# momentan avem 4 raspunsuri de la 0 la 3 facem cu I4 la celelalte mai speciale facem cu I8
print(y_train)
trg = [[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,0.0,1.0]]
y_train_target = []
for j in y_train:
  y_train_target.append(trg[j])
# y_1 = np.array(y_train_target)
# print(type(y_train_target[0][0]))

y_test = y_train_target[60:]
y_train = y_train_target[0:60]


[1, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 0, 1, 0, 3, 1, 1, 1, 0, 3, 2, 1, 1, 3, 1, 1, 1, 0, 1, 1, 3, 1, 3, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1, 2, 2, 1, 1, 3, 1, 0, 1, 1, 2, 2, 0, 1, 0, 0, 3, 1, 1, 0, 1, 1, 1, 3, 2, 3, 1]


In [ ]:
y_test = np.asarray(y_test)
y_train = np.asarray(y_train)

In [ ]:
history = model.fit(x_train_padded, y_train, epochs=20, validation_data=(x_test_padded, y_test))

In [ ]:
loss, accuracy = model.evaluate(x_test_padded,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

# **FINAL CNN**